In [1]:
%load_ext sql

In [2]:
import csv, sqlite3

con = sqlite3.connect("my_data1.db")
cur = con.cursor()

In [3]:
%sql sqlite:///my_data1.db

In [16]:
import pandas as pd
df = pd.read_csv("https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_2/data/Spacex.csv")
df.to_sql("SPACEXTBL", con, if_exists='replace', index=False,method="multi")
df.head()

,Date,Time (UTC),Booster_Version,Launch_Site,Payload,PAYLOAD_MASS__KG_,Orbit,Customer,Mission_Outcome,Landing_Outcome
0,2010-04-06,18:45:00,F9 v1.0 B0003,CCAFS LC-40,Dragon Spacecraft Qualification Unit,0,LEO,SpaceX,Success,Failure (parachute)
1,2010-08-12,15:43:00,F9 v1.0 B0004,CCAFS LC-40,"Dragon demo flight C1, two CubeSats, barrel of...",0,LEO (ISS),NASA (COTS) NRO,Success,Failure (parachute)
2,2012-05-22,07:44:00,F9 v1.0 B0005,CCAFS LC-40,Dragon demo flight C2,525,LEO (ISS),NASA (COTS),Success,No attempt
3,2012-08-10,00:35:00,F9 v1.0 B0006,CCAFS LC-40,SpaceX CRS-1,500,LEO (ISS),NASA (CRS),Success,No attempt
4,2013-01-03,15:10:00,F9 v1.0 B0007,CCAFS LC-40,SpaceX CRS-2,677,LEO (ISS),NASA (CRS),Success,No attempt


In [5]:
#remove blank rows from table
%sql create table SPACEXTABLE as select * from SPACEXTBL where Date is not null

 * sqlite:///my_data1.db
Done.


[]

In [10]:
#Display the names of the unique launch sites in the space mission
%sql select distinct (Launch_Site) from SPACEXTABLE

 * sqlite:///my_data1.db
Done.


Launch_Site
CCAFS LC-40
VAFB SLC-4E
KSC LC-39A
CCAFS SLC-40


In [9]:
#Display 5 records where launch sites begin with the string 'CCA'
%sql select "launch_site" from SPACEXTABLE where "launch_site" like 'CCA%' limit 5

 * sqlite:///my_data1.db
Done.


Launch_Site
CCAFS LC-40
CCAFS LC-40
CCAFS LC-40
CCAFS LC-40
CCAFS LC-40


In [18]:
#Display the total payload mass carried by boosters launched by NASA (CRS)
%sql select sum("PAYLOAD_MASS__KG_") as total_mass from SPACEXTABLE

 * sqlite:///my_data1.db
Done.


total_mass
619967


In [19]:
#Display average payload mass carried by booster version F9 v1.1
%sql select AVG("PAYLOAD_MASS__KG_") from SPACEXTABLE where "booster_version" like 'F9 v1.1%'

 * sqlite:///my_data1.db
Done.


AVG(PAYLOAD_MASS__KG_)
2534.6666666666665


In [23]:
#List the date when the first succesful landing outcome in ground pad was acheived.
%sql select min( "date" ) from SPACEXTABLE where "landing_outcome"== 'Success (ground pad)'

 * sqlite:///my_data1.db
Done.


min( date )
2015-12-22


In [26]:
#List the names of the boosters which have success in drone ship and have payload mass greater than 4000 but less than 6000
%sql select distinct("booster_version") from SPACEXTABLE where "landing_outcome" == 'Success (drone ship)' and "PAYLOAD_MASS__KG_" between 4000 and 6000

 * sqlite:///my_data1.db
Done.


Booster_Version
F9 FT B1022
F9 FT B1026
F9 FT B1021.2
F9 FT B1031.2


In [24]:
df['Landing_Outcome'].unique()

array(['Failure (parachute)', 'No attempt', 'Uncontrolled (ocean)',
       'Controlled (ocean)', 'Failure (drone ship)',
       'Precluded (drone ship)', 'Success (ground pad)',
       'Success (drone ship)', 'Success', 'Failure', 'No attempt '],
      dtype=object)

In [40]:
#List the total number of successful and failure mission outcomes
%sql select count (*) from SPACEXTABLE where  "landing_outcome" like '%Failure%'  or  "landing_outcome"  like '%Success%'

 * sqlite:///my_data1.db
Done.


count (*)
71


In [42]:
#List the names of the booster_versions which have carried the maximum payload mass. Use a subquery
%sql select distinct("booster_version") from SPACEXTABLE where "PAYLOAD_MASS__KG_" == (select max("PAYLOAD_MASS__KG_") from SPACEXTABLE)

 * sqlite:///my_data1.db
Done.


Booster_Version
F9 B5 B1048.4
F9 B5 B1049.4
F9 B5 B1051.3
F9 B5 B1056.4
F9 B5 B1048.5
F9 B5 B1051.4
F9 B5 B1049.5
F9 B5 B1060.2
F9 B5 B1058.3
F9 B5 B1051.6


In [52]:
#List the records which will display the month names, failure landing_outcomes in drone ship ,booster versions, 
#launch_site for the months in year 2015.
%sql select substr(Date, 6,2) as month, "landing_outcome", "booster_version", "launch_site" from SPACEXTABLE where "landing_outcome"== 'Failure (drone ship)' and substr(Date,0,5)='2015' 

 * sqlite:///my_data1.db
Done.


month,Landing_Outcome,Booster_Version,Launch_Site
10,Failure (drone ship),F9 v1.1 B1012,CCAFS LC-40
04,Failure (drone ship),F9 v1.1 B1015,CCAFS LC-40


In [60]:
#Rank the count of landing outcomes (such as Failure (drone ship) or Success (ground pad)) 
#between the date 2010-06-04 and 2017-03-20, in descending order.
                